## Unit 10. 線性模型回歸 課堂作業
- 課程編號: CHE-AI-114
- 學年度: 114下
- 上課時間: 每週四 09:10-12:00
- 上課地點: 學思樓 705
-
- 指導教授: 莊曜禎 助理教授
- 學生姓名: ＯＯＯ
- 學號: D12345678
- email帳號: fcu.d12345678@gmail.com

---
### 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P3_Unit10_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_3' / 'Unit10'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### 載入相關套件

In [ ]:
# 基礎套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# sklearn 套件
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 線性模型
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    SGDRegressor
)

# 設定隨機種子
SEED = 42
np.random.seed(SEED)

# 設定 matplotlib 中文顯示
plt.rcParams['axes.unicode_minus'] = False

print("✓ 套件載入完成！")

---
### **I. 練習題: 建立線性模型進行化工製程預測**
 - 使用下面化工反應器模擬數據集
 - 建立本單元所學的所有線性模型
 - 進行模型訓練與綜合評估比較

---
### **0. 數據生成**
生成化工反應器操作數據（聚合反應產率預測）

In [ ]:
# 設定隨機種子
np.random.seed(42)

# 生成特徵數據（10,000 筆樣本，10 個特徵）
n_samples = 10000
data = {
    'Temperature_C': np.random.uniform(150, 250, n_samples),           # 反應溫度 (°C)
    'Pressure_bar': np.random.uniform(1, 10, n_samples),               # 反應壓力 (bar)
    'Catalyst_wt%': np.random.uniform(0.1, 2.0, n_samples),            # 催化劑濃度 (wt%)
    'Residence_Time_min': np.random.uniform(5, 30, n_samples),         # 停留時間 (min)
    'Feed_Rate_L/hr': np.random.uniform(10, 50, n_samples),            # 進料流量 (L/hr)
    'pH': np.random.uniform(4, 10, n_samples),                         # pH 值
    'Stirring_Speed_rpm': np.random.uniform(100, 500, n_samples),      # 攪拌速度 (rpm)
    'Initiator_conc_M': np.random.uniform(0.01, 0.1, n_samples),       # 引發劑濃度 (M)
    'Coolant_Flow_L/min': np.random.uniform(5, 20, n_samples),         # 冷卻水流量 (L/min)
    'Feed_Composition': np.random.uniform(0.5, 1.5, n_samples)         # 進料組成
}

df = pd.DataFrame(data)

# 生成目標變數（產品產率 Yield_%）- 線性關係 + 噪聲
# 真實模型: Y = 150 + 15*T + 4*P + 1*C + 6*Rt - 5*Fr + 0.2*pH + 2.5*SS + 0.3*IC - 0.4*CF + 0.1*FC + noise
true_intercept = 150
true_coef = np.array([15, 4, 1, 6, -5, 0.2, 2.5, 0.3, -0.4, 0.1])

# 計算線性組合
X_array = df.values
yield_base = true_intercept + X_array @ true_coef

# 添加噪聲
noise = np.random.normal(0, 10, n_samples)
yield_rate = yield_base + noise

# 添加 5% 異常值（模擬傳感器故障）
n_outliers = int(0.05 * n_samples)
outlier_indices = np.random.choice(n_samples, n_outliers, replace=False)
yield_rate[outlier_indices] += np.random.choice([-1, 1], n_outliers) * np.random.uniform(30, 60, n_outliers)

df['Yield_%'] = yield_rate

# 保存數據
df.to_csv(DATA_DIR/'reactor_yield_data.csv', index=False, encoding='utf-8-sig')
print(f"數據集已生成: {DATA_DIR}/reactor_yield_data.csv")
print(f"數據形狀: {df.shape}")
print(f"\n前5筆數據:\n{df.head()}")
print(f"\n數據統計:\n{df.describe()}")

---
### **1. 數據準備與探索**
- 載入數據並檢查基本統計資訊
- 繪製目標變數（產率）分布圖
- 繪製至少 3 個重要特徵與目標變數的散點圖
- 計算特徵之間的相關係數矩陣
- 數據分割: 訓練集 80%、測試集 20%
- **使用 StandardScaler 對特徵進行標準化**

---
### **2. 建立所有線性模型**
建立以下 5 種線性模型:
1. **Linear Regression (OLS)**: 普通最小平方法
2. **Ridge Regression**: L2 正則化，alpha=1.0
3. **Lasso Regression**: L1 正則化，alpha=1.0
4. **Elastic Net**: L1+L2 混合正則化，alpha=1.0, l1_ratio=0.5
5. **SGD Regressor**: 隨機梯度下降，penalty='l2', alpha=0.0001

**要求**:
- 使用字典存儲所有模型
- 設定 random_state=42（如適用）
- 輸出每個模型的簡要說明

---
### **3. 模型訓練**
- 使用標準化後的訓練數據訓練所有模型
- 記錄每個模型的訓練時間
- 輸出訓練完成訊息

---
### **4. 模型評估與比較**

#### 4.1 計算評估指標
對每個模型計算以下指標:
- **Train R²**: 訓練集決定係數
- **Test R²**: 測試集決定係數
- **Test MAE**: 測試集平均絕對誤差
- **Test RMSE**: 測試集均方根誤差
- **Train Time**: 訓練時間（秒）

製作比較表格，並找出:
- 最佳 R² 模型
- 最佳 RMSE 模型
- 最快訓練速度模型

#### 4.2 係數比較
- 繪製所有模型的係數柱狀圖（每個模型一個子圖）
- 比較不同正則化方法對係數的影響
- 分析哪些特徵最重要

#### 4.3 預測結果視覺化
- 繪製所有模型的預測值 vs 實際值散點圖（5個子圖）
- 每個子圖包含理想線（y=x）
- 標註 R² 和 RMSE
- 分析哪個模型的預測最接近實際值

---
### **5. 交叉驗證**
- 對所有模型進行 5-Fold 交叉驗證
- 使用 R² 作為評分指標
- 比較交叉驗證分數與測試集分數
- 分析模型的穩定性

---
### **6. 超參數調整（擇一進行）**

選擇 Ridge、Lasso 或 Elastic Net 其中一個模型進行超參數調整:

#### 6.1 Ridge Regression 超參數調整
- alpha: [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

#### 6.2 Lasso Regression 超參數調整
- alpha: [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

#### 6.3 Elastic Net 超參數調整
- alpha: [0.001, 0.01, 0.1, 1.0, 10.0]
- l1_ratio: [0.1, 0.3, 0.5, 0.7, 0.9]

**要求**:
- 使用 GridSearchCV，cv=5
- 輸出最佳參數
- 比較調整前後的性能改善
- 繪製參數對性能的影響圖

---
### **7. 特徵重要性分析**
- 使用 Lasso 或 Elastic Net 進行特徵選擇
- 識別係數為 0 或接近 0 的特徵
- 繪製特徵重要性排序圖
- 分析哪些特徵對產率影響最大

---
### **8. 模型保存**
- 選擇表現最佳的模型
- 保存模型檔案（使用 joblib）
- 保存標準化器（StandardScaler）
- 保存模型資訊（JSON 格式）
  - 模型類型
  - 訓練/測試 R²
  - 測試 RMSE
  - 特徵名稱
  - 係數
  - 截距
  - 訓練日期

---
### **9. 綜合分析與結論**

請根據你的實驗結果回答以下問題:

1. **模型性能比較**:
   - 哪個模型表現最好？為什麼？
   - OLS 與正則化模型的性能差異如何？

2. **正則化效果**:
   - Ridge、Lasso、Elastic Net 的係數有何差異？
   - Lasso 是否成功進行特徵選擇？

3. **訓練效率**:
   - 哪個模型訓練最快？
   - SGD 與其他模型的訓練時間差異如何？

4. **化工應用啟示**:
   - 哪些操作變數對產率影響最大？
   - 如何利用模型優化反應器操作？

5. **模型選擇建議**:
   - 在此案例中，你會推薦使用哪個模型？為什麼？
   - 如果數據量增加到 100 萬筆，你會如何調整模型選擇？

---
## II. 額外加分作業 (自由繳交)
- 學生可自由選擇是否繳交加分作業（下週上課前完成 email 電子檔）
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI，問同學，問學長姊，甚至參考以前別人的作業，但一定要自行 [吸收][執行][整理] 結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容（與他人雷同，原創性低，抄襲比例過高），作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉（提供作業給別人複製的也一樣）
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

---
### 使用課堂作業所提供的化工製程模擬數據集
- 嘗試修改生成模擬數據的程式
- 改變參數、改變公式、添加特徵、添加非線性項...等等
- 產生獨一無二屬於你的數據集
- 建立線性模型來預測產品產率
- 然後開始進行下面實驗

---
### **實驗1: 正則化強度影響 (Ridge/Lasso)**

比較不同 alpha 值對模型性能的影響:
- alpha = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

**要求 (Ridge)**:
- 繪製 alpha vs R² 曲線
- 繪製 alpha vs RMSE 曲線
- 繪製係數隨 alpha 變化的軌跡圖
- **分析**: alpha 如何影響係數大小？最佳 alpha 是多少？

**要求 (Lasso)**:
- 繪製 alpha vs R² 曲線
- 繪製 alpha vs 非零係數數量曲線
- 繪製係數隨 alpha 變化的軌跡圖（Lasso Path）
- **分析**: alpha 如何影響特徵選擇？哪些特徵最先被剔除？

---
### **實驗2: Elastic Net 的 L1 Ratio 影響**

固定 alpha=1.0，比較不同 l1_ratio 的影響:
- l1_ratio = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

**要求**:
- 記錄每個 l1_ratio 的:
  - Test R²
  - Test RMSE
  - 非零係數數量
- 繪製 l1_ratio vs R² 曲線
- 繪製 l1_ratio vs 非零係數數量曲線
- **分析**: 
  - l1_ratio=0 相當於什麼模型？
  - l1_ratio=1 相當於什麼模型？
  - 最佳 l1_ratio 是多少？為什麼？

---
### **實驗3: 訓練數據量影響**

比較不同訓練數據量對模型性能的影響:
- 訓練樣本數 = [100, 500, 1000, 2000, 5000, 8000]

**要求**:
- 使用 OLS、Ridge、Lasso 三個模型
- 對每個數據量:
  - 隨機抽樣相應數量的訓練數據
  - 訓練模型
  - 在完整測試集上評估
  - 記錄 Test R² 和 Test RMSE
- 繪製學習曲線（Training Size vs R²）
- **分析**: 
  - 哪個模型在小數據集上表現更好？
  - 數據量多大時模型性能趨於穩定？
  - 正則化在小數據集上的作用如何？

---
### **實驗4: 多重共線性影響**

添加高度相關的特徵，觀察對模型的影響:

**要求**:
1. 創建新特徵:
   - `Temperature_C_squared` = Temperature_C²
   - `Temperature_C_cubed` = Temperature_C³
   - `Pressure_x_Temp` = Pressure_bar × Temperature_C
   - 這些特徵與原始特徵高度相關

2. 比較添加這些特徵前後:
   - OLS 的係數變化和性能變化
   - Ridge 的係數變化和性能變化
   - Lasso 的係數變化和性能變化

3. 計算特徵之間的相關係數矩陣

4. **分析**:
   - OLS 在多重共線性下有何問題？
   - Ridge 如何處理多重共線性？
   - Lasso 如何處理多重共線性？

---
### **實驗5: SGD 參數影響**

探討 SGD Regressor 的關鍵參數:

**5.1 學習率策略**
- learning_rate = ['constant', 'optimal', 'invscaling', 'adaptive']
- 比較收斂速度和最終性能

**5.2 最大迭代次數**
- max_iter = [10, 50, 100, 500, 1000, 2000]
- 繪製迭代次數 vs 性能曲線

**5.3 正則化類型**
- penalty = ['l2', 'l1', 'elasticnet', 'none']
- 比較不同正則化對係數的影響

**要求**:
- 記錄訓練時間
- 記錄模型性能
- **分析**: 
  - SGD 在什麼情況下優於批量梯度下降？
  - 如何選擇合適的學習率策略？
  - SGD 與其他線性模型的主要差異？

---
### 💭 思考題

1. **正則化理解**:
   - 為什麼 Ridge 不會將係數壓縮到 0，但 Lasso 可以？
   - 什麼情況下應該使用 Ridge？什麼情況下使用 Lasso？

2. **特徵標準化**:
   - 為什麼正則化模型必須進行特徵標準化？
   - 如果不標準化會有什麼問題？

3. **模型選擇**:
   - 在小數據集（< 1000 樣本）上，應該使用哪個模型？
   - 在大數據集（> 100 萬樣本）上，應該使用哪個模型？

4. **交叉驗證**:
   - 為什麼測試集 R² 與交叉驗證 R² 可能不同？
   - 哪個指標更能反映模型的真實性能？

5. **化工應用**:
   - 如果某個操作變數的係數為負，代表什麼意義？
   - 如何利用模型係數指導實際生產優化？

6. **過擬合與欠擬合**:
   - 如何判斷模型是否過擬合？
   - 正則化如何防止過擬合？

7. **SGD vs 批量梯度下降**:
   - SGD 的優勢是什麼？劣勢是什麼？
   - 在什麼情況下 SGD 是唯一選擇？

---
# 想對老師說的話